In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial.distance import euclidean
from colour.appearance import XYZ_to_CIECAM02
from math import cos, sin, sqrt, radians
from colour import SDS_ILLUMINANTS, SpectralShape
from scipy.interpolate import interp1d

In [5]:
def load_spd_excel_resample_to_5nm(path):
    df = pd.read_excel(path)
    df = df.apply(pd.to_numeric, errors='coerce')
    df = df.dropna()  

    wavelengths_1nm = df.iloc[:, 0].values 
    spd_values_1nm = df.iloc[:, 1].values  

    interp_func = interp1d(wavelengths_1nm, spd_values_1nm, kind='linear', bounds_error=False, fill_value=0)
    wavelengths_5nm = np.arange(380, 781, 5)
    spd_values_5nm = interp_func(wavelengths_5nm)

    return dict(zip(wavelengths_5nm, spd_values_5nm))


def load_d65_csv(path):
    df = pd.read_csv(path) 
    df = df.apply(pd.to_numeric, errors='coerce')  
    df = df.fillna(0)
    return dict(zip(df['Wavelength'], df['D65']))

    
def load_ces_dat(path):
    data = np.loadtxt(path, delimiter=',')
    wavelengths = data[:, 0]
    sample_values = data[:, 1:]
    ces_samples = []
    for i in range(sample_values.shape[1]):
        sample = dict(zip(wavelengths, sample_values[:, i]))
        ces_samples.append(sample)
    return ces_samples

def load_cmf_csv(path):
    df = pd.read_csv(path)
    # 取 380, 385, 390 ... 780的波长索引
    wavelengths = np.arange(380, 781, 5)
    cmf = {}
    for wl in wavelengths:
        i = wl - 380  # 因为df的行索引对应波长 - 380
        cmf[wl] = (df.loc[i, 'x_bar'], df.loc[i, 'y_bar'], df.loc[i, 'z_bar'])
    return cmf




In [6]:
def downsample_ces(ces_samples, step=5):
    downsampled = []
    for ces in ces_samples:  # 这里每个ces本身就是字典，不解包
        new_ces = {wl: ces[wl] for wl in range(380, 781, step)}
        downsampled.append(new_ces)
    return downsampled

In [7]:
def compute_XYZ(spd, reflectance, cmfs):
    wavelengths = np.arange(380, 781, 5)
    d_lambda = 5
    X = Y = Z = 0.0
    for wl in wavelengths:
        S = spd.get(wl, 0)
        R = reflectance.get(wl, 0)
        x_bar, y_bar, z_bar = cmfs.get(wl, (0, 0, 0))
        X += S * R * x_bar
        Y += S * R * y_bar
        Z += S * R * z_bar
    k = 100 / (sum(spd.get(wl, 0) * cmfs.get(wl, (0, 0, 0))[1] * d_lambda for wl in wavelengths))
    X *= k * d_lambda
    Y *= k * d_lambda
    Z *= k * d_lambda
    return [X, Y, Z]


In [8]:
def cam02_jmh_to_ab(J, M, h):
    h_rad = np.deg2rad(h)
    a = M * np.cos(h_rad)
    b = M * np.sin(h_rad)
    return a, b

In [9]:
def ciecam02_to_ucs(J, a, b):
    # 2006 CAM02-UCS conversion公式
    c1 = 0.007
    c2 = 0.0228
    c3 = 1.0

    Jp = (1 + 100 * c1) * J / (1 + c1 * J)
    ap = c2 * a
    bp = c2 * b
    return np.array([Jp, ap, bp])

In [74]:
def color_differences(jab1, jab2):
    return np.linalg.norm(jab1 - jab2, axis=1)

def compute_Rf_Rg(delta_Es, jab_test, jab_ref, max_ratio=5.0):
    Rf = 100 - 4.6 * np.mean(delta_Es)

    a_test = jab_test[:, 1]
    b_test = jab_test[:, 2]
    chroma_test = np.sqrt(a_test**2 + b_test**2)

    a_ref = jab_ref[:, 1]
    b_ref = jab_ref[:, 2]
    chroma_ref = np.sqrt(a_ref**2 + b_ref**2)

    eps = 1e-12
    chroma_ref_safe = np.where(chroma_ref < eps, eps, chroma_ref)

    ratios = chroma_test / chroma_ref_safe
    ratios_clipped = np.clip(ratios, None, max_ratio)

    Rg = 100 * np.mean(ratios_clipped)


    return Rf, Rg




In [75]:
def get_Rf_Rg_indices(spd_df):
    # Step 1: 将 DataFrame 转为 SPD 字典（自动重采样为5nm）
    wavelengths_1nm = spd_df.iloc[:, 0].values
    spd_values_1nm = spd_df.iloc[:, 1].values
    interp_func = interp1d(wavelengths_1nm, spd_values_1nm, kind='linear', bounds_error=False, fill_value=0)
    wavelengths_5nm = np.arange(380, 781, 5)
    spd_values_5nm = interp_func(wavelengths_5nm)
    spd_test = dict(zip(wavelengths_5nm, spd_values_5nm))

    # Step 2: 加载其他静态数据
    d65_def = load_d65_csv('d65.csv')
    ces_samples_1nm = load_ces_dat('IESTM30_15_Sspds.dat')
    ces_samples = downsample_ces(ces_samples_1nm)
    cmfs = load_cmf_csv('cmf.csv')

    # Step 3: 单位归一化（SPD和D65都按能量归一化）
    wavelengths = sorted(spd_test.keys())
    spd_array = np.array([spd_test[wl] for wl in wavelengths])
    d65_array = np.array([d65_def[wl] for wl in wavelengths])
    d65_scaled = d65_array * (np.sum(spd_array) / np.sum(d65_array))
    d65 = dict(zip(wavelengths, d65_scaled))

    # Step 4: 强度归一化（最大值归一化）
    max_spd = max(spd_test.values())
    spd_norm = {wl: val / max_spd for wl, val in spd_test.items()}
    max_d65 = max(d65_def.values())
    d65_norm = {wl: val / max_d65 for wl, val in d65_def.items()}

    # Step 5: 计算每个CES样本的 XYZ（测试光源 & 参考光源）
    XYZ_test_list = [compute_XYZ(spd_norm, sample, cmfs) for sample in ces_samples]
    XYZ_ref_list = [compute_XYZ(d65_norm, sample, cmfs) for sample in ces_samples]

    # Step 6: CIECAM02 变换
    XYZ_w = compute_XYZ(d65_norm, {wl: 1.0 for wl in range(380, 781, 5)}, cmfs)
    cam_test = [XYZ_to_CIECAM02(xyz, XYZ_w=XYZ_w, L_A=64, Y_b=20) for xyz in XYZ_test_list]
    cam_ref = [XYZ_to_CIECAM02(xyz, XYZ_w=XYZ_w, L_A=64, Y_b=20) for xyz in XYZ_ref_list]

    # Step 7: CAM02-UCS 转换
    jab_test = np.array([ciecam02_to_ucs(cam.J, *cam02_jmh_to_ab(cam.J, cam.M, cam.h)) for cam in cam_test])
    jab_ref = np.array([ciecam02_to_ucs(cam.J, *cam02_jmh_to_ab(cam.J, cam.M, cam.h)) for cam in cam_ref])



    # Step 8: 色差计算 & 指标输出
    delta_E = color_differences(jab_test, jab_ref)
    Rf, Rg = compute_Rf_Rg(delta_E, jab_test, jab_ref, max_ratio=5.0)

    return Rf, Rg


In [76]:
spd_df = pd.read_excel("P1_processed_data.xlsx")
Rf, Rg = get_Rf_Rg_indices(spd_df)
print(f"Rf = {Rf:.2f}, Rg = {Rg:.2f}")


Rf = 89.88, Rg = 192.18
